In [105]:
# Import required libraries
import pandas as pd
import dash
# import dash_html_components as html
# import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

from dash import html
from dash import dcc

In [108]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# names for charts
spacex_df['name_class'] = spacex_df['class'].apply(lambda x: str(x))

# print(spacex_df['Launch Site'].unique()) —> ['CCAFS LC-40' 'VAFB SLC-4E' 'KSC LC-39A' 'CCAFS SLC-40']

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[{'label': 'All Sites',      'value': 'ALL'},
                                             {'label': 'CCAFS LC-40',    'value': 'CCAFS LC-40'},
                                             {'label': 'VAFB SLC-4E',    'value': 'VAFB SLC-4E'},  
                                             {'label': 'KSC LC-39A',     'value': 'KSC LC-39A'}, 
                                             {'label': 'CCAFS SLC-40',   'value': 'CCAFS SLC-40'}],
                                    value='ALL',
                                    placeholder='Select a Launch Site here',
                                    searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    value=[min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(
                                    dcc.Graph(id='success-payload-scatter-chart')
                                ),
    ]
)

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown',      component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(
                    filtered_df, 
                    values='class', 
                    names='Launch Site', 
                    title='All Sites - Pie Chart')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        val = filtered_df.groupby('class').size() * 100.0 / len(filtered_df)

        fig = px.pie(
                    filtered_df, 
                    values=val,
                    names=['0', '1'],
                    title=f'{entered_site} - Pie Chart')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [
                  Input(component_id='site-dropdown',  component_property='value'), 
                  Input(component_id="payload-slider", component_property="value")
              ])
def get_scatter_chart(entered_site, entered_payload):
    lo = entered_payload[0]
    hi = entered_payload[1]
    
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= lo) & (spacex_df['Payload Mass (kg)'] <= hi)]
    
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, 
                         y="class", 
                         x="Payload Mass (kg)", 
                         color="Booster Version Category",
                         title='All Sites - Scatter Plot')
        return fig
        
    else:
        # return the outcomes piechart for a selected site
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df, 
                         y="class", 
                         x="Payload Mass (kg)", 
                         color="Booster Version Category",
                         title=f'{entered_site} - Scatter Plot')
        return fig


# Run the app
if __name__ == '__main__':
    app.run_server(port=8050)